In [1]:
# Setup Cell
import pickle

# Reload saved data
try:
    with open("saved_data.pkl", "rb") as f:
        data = pickle.load(f)
        print("Data loaded:", data)
except FileNotFoundError:
    print("No saved data found.")


No saved data found.


In [6]:
pip install kaggle

Note: you may need to restart the kernel to use updated packages.


In [1]:
import os
import shutil

# Path to the kaggle.json file after upload (assuming it's in the current working directory)
source = "kaggle.json"

# Destination path where Kaggle expects the kaggle.json file
destination = os.path.join(os.path.expanduser("~"), ".kaggle", "kaggle.json")

# Create the .kaggle directory if it doesn't exist
os.makedirs(os.path.dirname(destination), exist_ok=True)

# Move the kaggle.json file to the .kaggle directory
shutil.move(source, destination)

# Set permissions to secure the file
os.chmod(destination, 0o600)

print("kaggle.json file configured successfully.")


kaggle.json file configured successfully.


In [2]:
#API to fetch the dataset from kaggle
!kaggle datasets download -d kazanova/sentiment140
 

Dataset URL: https://www.kaggle.com/datasets/kazanova/sentiment140
License(s): other
sentiment140.zip: Skipping, found more recently modified local copy (use --force to force download)


In [3]:
 #extracting the compressed

In [4]:
from zipfile import ZipFile
dataset="sentiment140.zip"
with ZipFile(dataset,'r') as zip:
    zip.extractall()
    print("The dataset is extracted")

The dataset is extracted


In [5]:
#importing the dependencies

In [6]:
import numpy as np
import pandas as pd 
import re
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [7]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\YOGINI\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [8]:
#printing the stopwords in english
print(stopwords.words('english'))

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

In [72]:
#data processing 


In [10]:
#loading the data frame from csv file to pandas dataframe

In [11]:
twitter_data=pd.read_csv('training.1600000.processed.noemoticon.csv',encoding='ISO.8859.1')

In [12]:
#checking the number of rows and columns 

In [13]:
twitter_data.shape

(1599999, 6)

In [14]:
twitter_data.head()

,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, that's a bummer. You shoulda got David Carr of Third Day to do it. ;D"
0,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
1,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
2,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
3,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."
4,0,1467811372,Mon Apr 06 22:20:00 PDT 2009,NO_QUERY,joy_wolf,@Kwesidei not the whole crew


In [15]:
#naming the columns and reading the dataset again

In [16]:
column_names=['target','id','date','flag','user','text']
twitter_data=pd.read_csv('training.1600000.processed.noemoticon.csv',names=column_names,encoding='ISO.8859.1')

In [17]:
twitter_data.shape

(1600000, 6)

In [18]:
twitter_data.head()

,target,id,date,flag,user,text
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."


In [19]:
#counting the number of missing values in the dataset

In [20]:
twitter_data.isnull().sum()

target    0
id        0
date      0
flag      0
user      0
text      0
dtype: int64

In [21]:
#checking the distribution of target column

In [22]:
twitter_data['target'].value_counts()

target
0    800000
4    800000
Name: count, dtype: int64

In [23]:
#convert the target 4 to 1

In [24]:
twitter_data.replace({'target':{4:1}},inplace=True)

In [25]:
twitter_data['target'].value_counts()

target
0    800000
1    800000
Name: count, dtype: int64

In [26]:
#stemming=process of reducing a word to its root word

In [27]:
port_stem=PorterStemmer()

In [28]:
def stemming(content):
    stemmed_content=re.sub('[^a-zA-Z]',' ',content)
    stemmed_content=stemmed_content.lower()
    stemmed_content=stemmed_content.split()
    stemmed_content=[port_stem.stem(word) for word in stemmed_content if not word in stopwords.words('english') ]
    stemmed_content=' '.join(stemmed_content)
    return stemmed_content

In [29]:
twitter_data['stemmed_content']=twitter_data['text'].apply(stemming)

In [30]:
twitter_data.head()

,target,id,date,flag,user,text,stemmed_content
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t...",switchfoot http twitpic com zl awww bummer sho...
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...,upset updat facebook text might cri result sch...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...,kenichan dive mani time ball manag save rest g...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire,whole bodi feel itchi like fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all....",nationwideclass behav mad see


In [31]:
print(twitter_data['stemmed_content'])

0          switchfoot http twitpic com zl awww bummer sho...
1          upset updat facebook text might cri result sch...
2          kenichan dive mani time ball manag save rest g...
3                            whole bodi feel itchi like fire
4                              nationwideclass behav mad see
                                 ...                        
1599995                           woke school best feel ever
1599996    thewdb com cool hear old walt interview http b...
1599997                         readi mojo makeov ask detail
1599998    happi th birthday boo alll time tupac amaru sh...
1599999    happi charitytuesday thenspcc sparkschar speak...
Name: stemmed_content, Length: 1600000, dtype: object


In [32]:
print(twitter_data['target'])

0          0
1          0
2          0
3          0
4          0
          ..
1599995    1
1599996    1
1599997    1
1599998    1
1599999    1
Name: target, Length: 1600000, dtype: int64


In [33]:
#separating the data and label

In [34]:
x=twitter_data['stemmed_content'].values
y=twitter_data['target'].values

In [35]:
print(x)

['switchfoot http twitpic com zl awww bummer shoulda got david carr third day'
 'upset updat facebook text might cri result school today also blah'
 'kenichan dive mani time ball manag save rest go bound' ...
 'readi mojo makeov ask detail'
 'happi th birthday boo alll time tupac amaru shakur'
 'happi charitytuesday thenspcc sparkschar speakinguph h']


In [36]:
print(y)

[0 0 0 ... 1 1 1]


In [37]:
#spliting the data to training data and test data

In [38]:
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2,stratify=y,random_state=2)

In [39]:
print(x.shape,x_train.shape,x_test.shape)

(1600000,) (1280000,) (320000,)


In [40]:
print(x_train)

['watch saw iv drink lil wine' 'hatermagazin'
 'even though favourit drink think vodka coke wipe mind time think im gonna find new drink'
 ... 'eager monday afternoon'
 'hope everyon mother great day wait hear guy store tomorrow'
 'love wake folger bad voic deeper']


In [41]:
print(x_test)

['mmangen fine much time chat twitter hubbi back summer amp tend domin free time'
 'ah may show w ruth kim amp geoffrey sanhueza'
 'ishatara mayb bay area thang dammit' ...
 'destini nevertheless hooray member wonder safe trip' 'feel well'
 'supersandro thank']


In [42]:
#feature extraction=converting the textual data to numerical data

In [43]:
vectorizer=TfidfVectorizer()

In [44]:
x_train=vectorizer.fit_transform(x_train)
x_test=vectorizer.transform(x_test)

In [45]:
print(x_train)

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 9453092 stored elements and shape (1280000, 461488)>
  Coords	Values
  (0, 436713)	0.27259876264838384
  (0, 354543)	0.3588091611460021
  (0, 185193)	0.5277679060576009
  (0, 109306)	0.3753708587402299
  (0, 235045)	0.41996827700291095
  (0, 443066)	0.4484755317023172
  (1, 160636)	1.0
  (2, 109306)	0.4591176413728317
  (2, 124484)	0.1892155960801415
  (2, 407301)	0.18709338684973031
  (2, 129411)	0.29074192727957143
  (2, 406399)	0.32105459490875526
  (2, 433560)	0.3296595898028565
  (2, 77929)	0.31284080750346344
  (2, 443430)	0.3348599670252845
  (2, 266729)	0.24123230668976975
  (2, 409143)	0.15169282335109835
  (2, 178061)	0.1619010109445149
  (2, 150715)	0.18803850583207948
  (2, 132311)	0.2028971570399794
  (2, 288470)	0.16786949597862733
  (3, 406399)	0.29029991238662284
  (3, 158711)	0.4456939372299574
  (3, 151770)	0.278559647704793
  (3, 56476)	0.5200465453608686
  :	:
  (1279996, 318303)	0.21254698865277744
  (12

In [46]:
print(x_test)

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 2289192 stored elements and shape (320000, 461488)>
  Coords	Values
  (0, 15110)	0.1719352837797837
  (0, 31168)	0.1624772418052177
  (0, 67828)	0.26800375270827315
  (0, 106069)	0.36555450010904555
  (0, 132364)	0.255254889555786
  (0, 138164)	0.23688292264071406
  (0, 171378)	0.2805816206356074
  (0, 271016)	0.45356623916588285
  (0, 279082)	0.17825180109103442
  (0, 388348)	0.2198507607206174
  (0, 398906)	0.34910438732642673
  (0, 409143)	0.3143047059807971
  (0, 420984)	0.17915624523539805
  (1, 6463)	0.30733520460524466
  (1, 15110)	0.211037449588008
  (1, 145393)	0.575262969264869
  (1, 217562)	0.40288153995289894
  (1, 256777)	0.28751585696559306
  (1, 348135)	0.4739279595416274
  (1, 366203)	0.24595562404108307
  (2, 22532)	0.3532582957477176
  (2, 34401)	0.37916255084357414
  (2, 89448)	0.36340369428387626
  (2, 183312)	0.5892069252021465
  (2, 256834)	0.2564939661498776
  :	:
  (319994, 443794)	0.2782185641032538


In [47]:
#training the machine learning model

In [49]:
#logistic regression=classification model can be used to classify the data points into different classes 

In [50]:
#in this case the two data points we have are positive tweets and negative tweets

In [51]:
model=LogisticRegression(max_iter=1000)

In [52]:
model.fit(x_train,y_train)

LogisticRegression(max_iter=1000)

In [53]:
#model evaluation(we are using accuracy score)

In [54]:
#accuracy score on the training data

In [55]:
x_train_prediction=model.predict(x_train)
training_data_accuracy=accuracy_score(y_train,x_train_prediction)

In [56]:
print('Accuracy score on the training data:',training_data_accuracy)

Accuracy score on the training data: 0.79871953125


In [57]:
x_test_prediction=model.predict(x_test)
test_data_accuracy=accuracy_score(y_test,x_test_prediction)

In [58]:
print('Accuracy score on the testing data:',test_data_accuracy)

Accuracy score on the testing data: 0.77668125


In [60]:
#model accuracy=77.6
#saving the trained model

In [62]:
import pickle

In [63]:
filename='trained_model.sav'
pickle.dump(model,open(filename,"wb"))

In [64]:
#using a saved model for future predictions

In [65]:
#loading the saved model

In [67]:
loaded_model=pickle.load(open('trained_model.sav','rb'))

In [69]:
x_new=x_test[200]
print(y_test[200])
prediction=loaded_model.predict(x_new)
print(prediction)

if(prediction[0]==0):
    print('Negative Tweet')

else:
    print('Positive Tweet')

1
[1]
Positive Tweet


In [71]:
x_new=x_test[3]
print(y_test[3])
prediction=loaded_model.predict(x_new)
print(prediction)

if(prediction[0]==0):
    print('Negative Tweet')

else:
    print('Positive Tweet')

0
[0]
Negative Tweet
